Code to deploy Llama3: https://www.analyticsvidhya.com/blog/2024/04/how-to-run-llama-3-locally/

## Install Dependencies

In [ ]:
!pip install -U "transformers==4.40.0" --upgrade
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install accelerate

## Download Llama 3

In [ ]:
import transformers
import torch

model_id = "unsloth/llama-3-8b-Instruct-bnb-4bit"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True},
        "low_cpu_mem_usage": True,
    },
)

## Query Llama 3

In [3]:
fallacies = """
Fallacy of Credibility: Abusive Ad Hominem; Ad Populum; Appeal to False Authority;
Appeal to Nature; Appeal to Tradition; Guilt by Association; Tu Quoque.
Fallacy of Logic: Causal Oversimplification; Circular Reasoning; Equivocation;
False Analogy; False Causality; False Dilemma; Hasty Generalization; Slippery Slope; Straw Man; Fallacy of Division.
Appeal to Emotion: Appeal to Positive Emotion; Appeal to Anger; Appeal to Fear;
Appeal to Pity; Appeal to Ridicule; Appeal to Worse Problem.
"""

In [17]:
fallacies_with_examples = """
Slippery slope:\n\nIf we ban assault rifles, next they'll ban all guns, and before you know it, we'll be living in a police state.,
Hasty generalization:\n\nI met two people from New York City, and both were rude. Therefore, all New Yorkers are rude.,
False analogy:\n\nComparing buying a car to choosing a spouse is a false analogy because the two situations are vastly different.,
Guilt by association:\n\nYou shouldn't listen to John's opinion on politics; he's friends with several criminals.,
Causal oversimplification:\n\nThe rise in crime rates is due to violent video games; therefore, banning them will solve the problem.,
Ad populum:\n\nEveryone is buying the new smartphone, so it must be the best one on the market.,
Circular reasoning:\n\nThe Bible is the word of God because God says it is in the Bible.,
Appeal to fear:\n\nIf you don't vote for me, your taxes will skyrocket and crime will run rampant.,
False causality:\n\nEvery time I wash my car, it rains. Therefore, washing my car causes rain.,
Fallacy of division:\n\nThat company is profitable, so all of its employees must be well-paid.,
Appeal to ridicule:\n\nYou shouldn't listen to Mary's argument about climate change; she believes the Earth is flat!,
Appeal to worse problems:\n\nWhy are we worrying about pollution when there are children starving in the world?,
Abusive ad hominem:\n\nYou can't trust anything Tom says; he's just a loser.,
Appeal to false authority:\n\nMy doctor says that smoking is good for you, so it must be true.,
Appeal to positive emotion:\n\nIf you care about the environment, you'll buy this eco-friendly product.,
Appeal to nature:\n\nNatural remedies are always better than pharmaceutical drugs.,
Equivocation:\n\nThe sign said 'Fine for parking here,' so I thought it would be fine to park there.,
Appeal to anger:\n\nYou wouldn't want to be a coward, would you? Then you better join the military!,
False dilemma:\n\nEither we ban all guns or we'll continue to have mass shootings.,
Straw man:\n\nOpponents of gun control argue that we should give guns to everyone, including criminals and children.,
Appeal to tradition:\n\nWe should continue to use fossil fuels because that's how we've always done it.,
Tu quoque:\n\nYou say I shouldn't smoke, but I've seen you smoke before!
"""

### Version №1: Vanilla zero-shot version of ToT prompting.

In [20]:
def tot_prompt_1(input_text):
    messages = [
        {"role"   : "system",
        "content": "You are a helpful assistant!"},
        {"role"   : "user",
        "content":
        f"As a content reviewer, I provide multiple fallacies which might occur in the text; \
        you need to identify the falacies if there're such in the text.\
        {fallacies}\
        Text: {input_text}\
        Walk me through this context in manageable parts step by step, \
        summarizing and analyzing as we go.\
        Answer:\
        "},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    return outputs[0]["generated_text"][len(prompt):]

In [ ]:
# The answer is "false analogy", model is CORRECT
print(tot_prompt_1(input_text=
             """
             TITLE: There is a difference between a'smurf' and an'alt'.
             Please learn it and stop using them interchangeably. POST: Someone once told me
             they have an'alt' cause their main account was too high of rank to play with their
             friends. It's exactly the same as smurfing.
             """))

I'd be happy to help you identify potential fallacies in the text.

**Step 1: Summary**
The text is about the difference between "smurf" and "alt". The author is responding to someone who mentioned having an "alt" because their main account was too high-rank to play with friends, and the author claims it's the same as "smurfing".

**Step 2: Analysis**
Let's break down the text into smaller parts:

* "Someone once told me they have an'alt' cause their main account was too high of rank to play with their friends."
	+ This statement appears to be an anecdote, but it could be an Appeal to Authority fallacy if the person who told the author is not a credible source.
* "It's exactly the same as smurfing."
	+ This statement seems to be an opinion or a comparison, but it could be a False Analogy fallacy if the author is not providing a valid comparison between "smurfing" and having an "alt".
* The author's tone is instructive, which could be an Appeal to Emotion (Appeal to Ridicule) if they're

In [ ]:
# The answer is "circular reasoning", model is NOT CORRECT
print(tot_prompt_1(input_text=
             """
             America is the best place to live, because it's better than any other country.
             """))

Let's break down the text and identify any potential fallacies.

**Initial Statement:** "America is the best place to live, because it's better than any other country."

**Step 1: Identify the Claim**

The claim is that America is the best place to live. The speaker is making a statement about the superiority of America compared to other countries.

**Step 2: Identify the Reason**

The reason given for this claim is that "it's better than any other country." This is a very general and vague statement, which raises some red flags.

**Step 3: Analyze the Reason**

The reason given is an example of a **Fallacy of Credibility: Appeal to False Authority**. The speaker is implying that America is the best place to live simply because it's better than other countries, without providing any evidence or specific reasons to support this claim. This is a weak and unconvincing argument, as it relies on an unsubstantiated claim rather than providing actual evidence.

**Step 4: Identify any other po

In [ ]:
# The answer is "causal oversimplification" and "false causality", model is NOT CORRECT
print(tot_prompt_1(input_text=
             """
             The last Democrat winner of the New Hampshire primary won the general election.
             This year, the winner of the New Hampshire primary will win the general election.
             """))

I'd be happy to help you identify any fallacies in the given text. Let's break it down step by step.

**Step 1: Identify the text**
The text consists of two sentences:

1. "The last Democrat winner of the New Hampshire primary won the general election."
2. "This year, the winner of the New Hampshire primary will win the general election."

**Step 2: Analyze the text**

The text appears to be making a claim about the relationship between winning the New Hampshire primary and winning the general election. The claim is based on the experience of the last Democrat winner of the New Hampshire primary, who allegedly won the general election.

**Step 3: Identify the fallacy**

The fallacy present in this text is the:

* **Appeal to Unrepresentative Sample** (also known as the Fallacy of Credibility: Guilt by Association)

The text assumes that the experience of the last Democrat winner of the New Hampshire primary is representative of all future winners. However, this is an unjustified assump

### Version №2: Vanilla zero-shot version of ToT prompting with modified output.

In [33]:
def tot_prompt_2(input_text):
    messages = [
            {"role"   : "system",
            "content": "You are a helpful assistant!"},
            {"role"   : "user",
            "content":
            f"As a content reviewer, I provide multiple fallacies which might occur in the text; \
            you need to identify the falacies if there're such in the text, else return 'no fallacy'.\
            {fallacies}\
            Text: {input_text}\
            Walk me through this context in manageable parts step by step, \
            summarizing and analyzing as we go. \
            Then, output just the identified fallacies on the last line separated by ';'.\
            Answer:\
            "},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )

    return outputs[0]["generated_text"][len(prompt):]

In [9]:
# The answer is "false analogy", model is CORRECT
print(tot_prompt_2(input_text=
             """
             TITLE: There is a difference between a'smurf' and an'alt'.
             Please learn it and stop using them interchangeably. POST: Someone once told me
             they have an'alt' cause their main account was too high of rank to play with their
             friends. It's exactly the same as smurfing.
             """))

Let's break down the text into manageable parts and analyze it step by step.

**Part 1:** TITLE: There is a difference between a'smurf' and an 'alt'.

* The title sets the tone for the discussion, claiming that there is a difference between two terms. However, it doesn't provide any context or explanation about what these terms mean or why they are different.

**Part 2:** POST: Someone once told me they have an 'alt' cause their main account was too high of rank to play with their friends. It's exactly the same as smurfing.

* The poster shares a personal anecdote about someone having an 'alt' account because their main account was too high in rank to play with friends. They then claim that this is the same as smurfing.

**Analysis:**

* The poster is trying to establish a connection between having an 'alt' account and smurfing, but the connection is not clearly explained. It seems like the poster is making an assumption or drawing a conclusion without providing sufficient evidence.
* 

In [13]:
# The answer is "circular reasoning", model is NOT CORRECT
print(tot_prompt_2(input_text=
             """
             America is the best place to live, because it's better than any other country.
             """))

Let's break down the text step by step and analyze it.

**Step 1: Identify the claim**
The claim is "America is the best place to live, because it's better than any other country."

**Step 2: Identify the reasoning**
The reasoning is simply "because it's better than any other country." This is a very weak and vague justification.

**Step 3: Analyze the reasoning**
The reasoning is an example of a **Fallacy of Credibility: Appeal to False Authority**. The speaker is implying that America is the best place to live simply because it's better than other countries, without providing any evidence or credible sources to support this claim. This is a false authority because it's not a reliable or credible source.

**Step 4: Identify any emotional appeals**
There is no apparent emotional appeal in this text.

**Step 5: Identify any logical fallacies**
There is a **Fallacy of Logic: False Causality**. The speaker is implying that being better than other countries is the cause of America being th

In [15]:
# The answer is "causal oversimplification" and "false causality", model is NOT CORRECT
print(tot_prompt_2(input_text=
             """
             The last Democrat winner of the New Hampshire primary won the general election.
             This year, the winner of the New Hampshire primary will win the general election.
             """))

Let's break down the text step by step:

1. The first sentence states: "The last Democrat winner of the New Hampshire primary won the general election."

This sentence is making a factual claim about the past. It's stating that the winner of the New Hampshire primary in the past (specifically, the last Democrat winner) went on to win the general election.

2. The second sentence states: "This year, the winner of the New Hampshire primary will win the general election."

This sentence is making a prediction about the future. It's stating that the winner of the New Hampshire primary this year will also go on to win the general election.

The connection between the two sentences is that the speaker is trying to draw a conclusion about the future based on the past. Specifically, they're saying that because the last Democrat winner of the New Hampshire primary won the general election, this year's winner will do the same.

However, this conclusion relies on an assumption that may not be val

In [29]:
# The answer is "slippery slope", model is NOT CORRECT
print(tot_prompt_2(input_text=
             """
             TITLE: Gatineau police officer tests positive, but masks still not mandatory at checkpoints POST:
             Guess what? police need to interact with people at close quarters. they often need to touch or grapple.
             The masks do little. Cool it. Once they wear masks your anxiety will force you to demand the next step.
             biohazard suits? get real.
             """))

Let's break down the text step by step:

**Step 1:** The title mentions a Gatineau police officer testing positive for COVID-19, but masks are not mandatory at checkpoints.

**Step 2:** The author starts by stating that police officers need to interact with people at close quarters, which implies that they are at risk of contracting COVID-19.

**Step 3:** The author claims that masks do little to prevent the spread of COVID-19, and that people will become anxious and demand more drastic measures (such as biohazard suits) if masks are worn.

**Step 4:** The author seems to be dismissing the idea of wearing masks, implying that it's an overreaction.

Now, let's analyze the text:

* The author is using a personal anecdote ("Guess what?") to grab the reader's attention, which could be seen as an attempt to establish credibility (Fallacy of Credibility: Appeal to Personal Experience).
* The author is downplaying the effectiveness of masks, which could be seen as an Appeal to Fear (Fallacy o

In [31]:
# The answer is "appeal to nature", model is CORRECT
print(tot_prompt_2(input_text=
             """
             TITLE: Sweden sees higher coronavirus death rate than US after refusing lockdown POST:
             I wonder how is New York planning on getting out of the lockdown,
             because when they get out of lockdown probably some cases are going to be going
             on and the whole infection thing will start again. POST: You cant stop nature.
             Youre going to be infected if you havent already. We are always trying to fight nature.
             We just have to let it run its course.
             """))

Let's break down the context step by step.

**Part 1:**
The original post wonders how New York will get out of the lockdown, suggesting that when the lockdown is lifted, the infection will start again.

**Part 2:**
The second post responds, saying that you can't stop nature and that people will get infected if they haven't already. The post also mentions that humans are always trying to fight nature and that we just need to let it run its course.

**Analysis:**

* The second post is using an Appeal to Nature fallacy, suggesting that nature is unstoppable and that humans should just accept it.
* The post is also using a False Analogy fallacy, comparing the natural world to a force that can't be controlled, when in reality, humans have successfully controlled and mitigated the spread of diseases through public health measures.
* Additionally, the post is using a Tu Quoque fallacy, implying that the first post is trying to fight nature, which is not a relevant or accurate criticism.

**Id

In [35]:
# The answer is "no fallacy", model is NOT CORRECT
print(tot_prompt_2(input_text=
             """
             The sea-ice loss is President Vladimir Putin' s gain.
             Already the largest country on the planet, Russia stands to gain
             access to shipping routes and energy reserves, and a strategic
             military advantage from the opening of the Arctic. Along the Russian
             coastline, which makes up more than half the Arctic total, winds
             and currents push old ice away from potential shipping lanes and
             prevent the build-up of thicker, multi-year ice that would leave
             other parts of the Arctic impassable for longer periods.
             """))

Let's break down the text into manageable parts and analyze it step by step.

**Part 1:** "The sea-ice loss is President Vladimir Putin's gain."

* This statement seems to be making a connection between the sea-ice loss and President Putin's gain. However, it's not clear what specific benefit Putin would gain from the sea-ice loss.

**Part 2:** "Already the largest country on the planet, Russia stands to gain access to shipping routes and energy reserves, and a strategic military advantage from the opening of the Arctic."

* This statement is making a claim about the benefits Russia would gain from the opening of the Arctic. However, it's not clear what specific evidence or reasoning is being used to support this claim.

**Part 3:** "Along the Russian coastline, which makes up more than half the Arctic total, winds and currents push old ice away from potential shipping lanes and prevent the build-up of thicker, multi-year ice that would leave other parts of the Arctic impassable for lo

### Version №3: Vanilla few-shot version of ToT prompting.

In [66]:
def tot_prompt_3(input_text):
    messages = [
        {"role"   : "system",
            "content": "You are an expert in detecting logical fallacies!"},
            {"role"   : "user",
            "content":
            f"As a content reviewer, I provide multiple fallacies and their examples which might occur in the text; \
            you need to identify the falacies if there're such in the text, else return 'no fallacy'.\
            {fallacies_with_examples}\
            Text: {input_text}\
            Walk me through this context in manageable parts step by step, \
            summarizing and analyzing as we go.\
            Then, provide only fallacies separated by ';' or the 'no fallacy' label.\
            Answer:\
            "},
    ]
    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )

    return outputs[0]["generated_text"][len(prompt):]

In [22]:
# The answer is "false analogy", model is CORRECT
print(tot_prompt_3(input_text=
             """
             TITLE: There is a difference between a'smurf' and an'alt'.
             Please learn it and stop using them interchangeably. POST: Someone once told me
             they have an'alt' cause their main account was too high of rank to play with their
             friends. It's exactly the same as smurfing.
             """))

Let's break down the text step by step and analyze it.

**Step 1:** The text is discussing a difference between a "smurf" and an "alt".

**Step 2:** The author mentions someone telling them that they have an "alt" because their main account was too high of rank to play with their friends. The author argues that this is the same as smurfing.

**Analysis:** The author is trying to make a point about the difference between a "smurf" and an "alt", but they are using a flawed analogy to do so. The situation described is not equivalent to smurfing, which is typically used to describe a player switching between multiple characters to gain an unfair advantage. The author's example is more about a player having a secondary account to play with friends, which is not the same thing.

**Fallacies identified:** Tu quoque; False analogy

The text contains a Tu quoque fallacy because the author is attacking the person who told them about the "alt" by saying that their example is not the same as smurf

In [24]:
# The answer is "circular reasoning", model is NOT CORRECT
print(tot_prompt_3(input_text=
             """
             America is the best place to live, because it's better than any other country.
             """))

Let's break down the text step by step:

1. The statement claims that "America is the best place to live" without providing any evidence or justification.

Summary: The statement makes a bold claim about America being the best place to live, but it lacks any supporting evidence or reasoning.

Analysis: The statement appears to be a subjective opinion or a value judgment, and it's not clear what criteria are being used to determine what makes a place "the best." This lack of specificity and evidence makes it difficult to evaluate the claim.

2. The statement compares America to "any other country," implying that there is a direct comparison between America and all other countries.

Summary: The statement is making a comparison between America and other countries, but it's not clear what specific aspects of these countries are being compared.

Analysis: The statement may be committing a false analogy fallacy, as the comparison between America and other countries is not specified or justi

In [27]:
# The answer is "causal oversimplification" and "false causality", model is NOT CORRECT
print(tot_prompt_3(input_text=
             """
             The last Democrat winner of the New Hampshire primary won the general election.
             This year, the winner of the New Hampshire primary will win the general election.
             """))

Let's break down the text step by step.

**Step 1: Identify the claim**
The claim is: "This year, the winner of the New Hampshire primary will win the general election."

**Step 2: Analyze the claim**
The claim is making a prediction about the outcome of the general election based on the outcome of the New Hampshire primary. However, there is no logical connection between the two. The claim is relying on a single data point (the last Democrat winner of the New Hampshire primary winning the general election) to make a prediction about a future event.

**Step 3: Identify the fallacy**
The fallacy committed here is **Hasty generalization**. The claim is making a generalization about the outcome of the general election based on a single data point, without considering other relevant factors that might affect the outcome.

**Step 4: Output the identified fallacy**
The identified fallacy is: Hasty generalization.

So, the output is:
Hasty generalization


In [28]:
# The answer is "slippery slope", model is NOT CORRECT
print(tot_prompt_3(input_text=
             """
             TITLE: Gatineau police officer tests positive, but masks still not mandatory at checkpoints POST:
             Guess what? police need to interact with people at close quarters. they often need to touch or grapple.
             The masks do little. Cool it. Once they wear masks your anxiety will force you to demand the next step.
             biohazard suits? get real.
             """))

Let's break down the text step by step.

**Part 1:** The author starts by stating that police officers need to interact with people at close quarters, which requires physical contact.

**Summary:** The author is setting the context for why police officers might not need to wear masks.

**Analysis:** So far, the text is providing a factual context, and there is no fallacy detected.

**Part 2:** The author claims that masks do little to prevent the spread of disease.

**Summary:** The author is expressing skepticism about the effectiveness of masks.

**Analysis:** This statement is an opinion, but it's not a fallacy yet.

**Part 3:** The author warns that if people wear masks, they will become anxious and demand more protection, such as biohazard suits.

**Summary:** The author is predicting a hypothetical scenario.

**Analysis:** This statement is an opinion, and it's where the fallacy might be hiding.

**Part 4:** The author concludes by telling the reader to "cool it" and not demand m

In [30]:
# The answer is "appeal to nature", model is CORRECT
print(tot_prompt_3(input_text=
             """
             TITLE: Sweden sees higher coronavirus death rate than US after refusing lockdown POST:
             I wonder how is New York planning on getting out of the lockdown,
             because when they get out of lockdown probably some cases are going to be going
             on and the whole infection thing will start again. POST: You cant stop nature.
             Youre going to be infected if you havent already. We are always trying to fight nature.
             We just have to let it run its course.
             """))

Let's break down the text step by step:

**Post 1**
"I wonder how is New York planning on getting out of the lockdown, because when they get out of lockdown probably some cases are going to be going on and the whole infection thing will start again."

* This post is making a prediction about the outcome of New York's lockdown. The speaker is wondering how they will exit the lockdown, implying that the current measures are not effective.
* This is a hasty generalization, as the speaker is making a broad statement about the effectiveness of lockdowns based on a single example (New York).

**Post 2**
"You can't stop nature. You're going to be infected if you haven't already. We are always trying to fight nature. We just have to let it run its course."

* This post is making a claim that it's impossible to stop the spread of the virus, and that it's natural for people to get infected.
* This is an appeal to nature, as the speaker is arguing that the spread of the virus is a natural phenome

In [32]:
# The answer is "no fallacy", model is CORRECT
print(tot_prompt_3(input_text=
             """
             The sea-ice loss is President Vladimir Putin' s gain.
             Already the largest country on the planet, Russia stands to gain
             access to shipping routes and energy reserves, and a strategic
             military advantage from the opening of the Arctic. Along the Russian
             coastline, which makes up more than half the Arctic total, winds
             and currents push old ice away from potential shipping lanes and
             prevent the build-up of thicker, multi-year ice that would leave
             other parts of the Arctic impassable for longer periods.
             """))

Let's break down the text into manageable parts and analyze it step by step.

**Part 1:** "The sea-ice loss is President Vladimir Putin's gain."

* Summary: The text claims that the loss of sea ice is beneficial to President Putin.
* Analysis: This statement is a vague and unsubstantiated claim. It's unclear what specific benefits Putin would gain from sea ice loss.

**Part 2:** "Already the largest country on the planet, Russia stands to gain access to shipping routes and energy reserves..."

* Summary: The text claims that Russia will gain access to shipping routes and energy reserves due to sea ice loss.
* Analysis: This statement is a claim about the benefits of sea ice loss, but it's not clear how this would occur or what the evidence is for this claim.

**Part 3:** "...and a strategic military advantage from the opening of the Arctic."

* Summary: The text claims that sea ice loss would give Russia a strategic military advantage in the Arctic.
* Analysis: This statement is anothe

### Version №4: Advanced zero-shot version of ToT prompting: two-step prompting using reasoning + refining conclusion.

In [65]:
def tot_prompt_4(input_text):
    # Step 1: Initial reasoning
    messages = [
        {"role": "system", "content": "You are a helpful assistant!"},
        {"role": "user", "content":
            f"As a content reviewer, I provide multiple fallacies which might occur in the text; \
            you need to identify the fallacies if there're such in the text, else return 'no fallacy'.\
            {fallacies}\
            Text: {input_text}\
            Walk me through this context in manageable parts step by step, \
            summarizing and analyzing as we go. \
            Answer:\
            "}
    ]

    # Generate the initial analysis using the pipeline
    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )

    initial_analysis = outputs[0]["generated_text"][len(prompt):]

    # Step 2: Refining the conclusion
    messages_refine = [
        {"role": "system", "content": "You are a helpful assistant!"},
        {"role": "user", "content":
            f"Based on the analysis provided, distill the findings into a concise summary. \
            For each identified fallacy, specify the text segment and the corresponding fallacy type. \
            Finally, provide only fallacies separated by ';' or the 'no fallacy' label.\
            Summarize the identified fallacies and their types from the initial analysis:\
            {initial_analysis}\
            Provide only the identified fallacies."
        }
    ]

    prompt_refine = pipeline.tokenizer.apply_chat_template(
        messages_refine,
        tokenize=False,
        add_generation_prompt=True
    )

    outputs_refine = pipeline(
        prompt_refine,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )

    refined_output = outputs_refine[0]["generated_text"][len(prompt_refine):]

    return refined_output

In [50]:
# The answer is "false analogy", model is CORRECT
print(tot_prompt_4(input_text=
             """
             TITLE: There is a difference between a'smurf' and an'alt'.
             Please learn it and stop using them interchangeably. POST: Someone once told me
             they have an'alt' cause their main account was too high of rank to play with their
             friends. It's exactly the same as smurfing.
             """))

Here is the concise summary:

Appeal to False Authority; False Analogy; Causal Oversimplification;


In [51]:
# The answer is "circular reasoning", model is NOT CORRECT
print(tot_prompt_4(input_text=
             """
             America is the best place to live, because it's better than any other country.
             """))

Appeal to Ignorance; False Dilemma


In [56]:
# The answer is "no fallacy", model is NOT CORRECT
print(tot_prompt_4(input_text=
             """
             The sea-ice loss is President Vladimir Putin' s gain.
             Already the largest country on the planet, Russia stands to gain
             access to shipping routes and energy reserves, and a strategic
             military advantage from the opening of the Arctic. Along the Russian
             coastline, which makes up more than half the Arctic total, winds
             and currents push old ice away from potential shipping lanes and
             prevent the build-up of thicker, multi-year ice that would leave
             other parts of the Arctic impassable for longer periods.
             """))

Here are the identified fallacies:

Appeal to False Authority; Appeal to Fear; False Causality; Hasty Generalization;


### Version №5: Advanced few-shot version of ToT prompting: two-step prompting using reasoning + refining conclusion.

In [63]:
def tot_prompt_5(input_text):
    # Step 1: Initial reasoning
    messages = [
        {"role": "system", "content": "You are a helpful assistant!"},
        {"role": "user", "content":
            f"As a content reviewer, I provide multiple fallacies and their examples which might occur in the text; \
            you need to identify the fallacies if there're such in the text, else return 'no fallacy'.\
            {fallacies_with_examples}\
            Text: {input_text}\
            Walk me through this context in manageable parts step by step, \
            summarizing and analyzing as we go. \
            Answer:\
            "}
    ]

    # Generate the initial analysis using the pipeline
    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )

    initial_analysis = outputs[0]["generated_text"][len(prompt):]

    # Step 2: Refining the conclusion
    messages_refine = [
        {"role": "system", "content": "You are a helpful assistant!"},
        {"role": "user", "content":
            f"Based on the analysis provided, distill the findings into a concise summary. \
            For each identified fallacy, specify the text segment and the corresponding fallacy type. \
            Finally, provide only fallacies separated by ';' or the 'no fallacy' label.\
            Summarize the identified fallacies and their types from the initial analysis:\
            {initial_analysis}\
            Provide only the identified fallacies."
        }
    ]

    prompt_refine = pipeline.tokenizer.apply_chat_template(
        messages_refine,
        tokenize=False,
        add_generation_prompt=True
    )

    outputs_refine = pipeline(
        prompt_refine,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )

    refined_output = outputs_refine[0]["generated_text"][len(prompt_refine):]

    return refined_output

In [60]:
# The answer is "false analogy", model is CORRECT
print(tot_prompt_5(input_text=
             """
             TITLE: There is a difference between a'smurf' and an'alt'.
             Please learn it and stop using them interchangeably. POST: Someone once told me
             they have an'alt' cause their main account was too high of rank to play with their
             friends. It's exactly the same as smurfing.
             """))

Here are the identified fallacies, separated by semicolons:

False analogy


In [67]:
# The answer is "circular reasoning", model is NOT CORRECT
print(tot_prompt_5(input_text=
             """
             America is the best place to live, because it's better than any other country.
             """))

Here are the identified fallacies separated by ';':

False dichotomy; False analogy


In [68]:
# The answer is "no fallacy", model is NOT CORRECT
print(tot_prompt_5(input_text=
             """
             The sea-ice loss is President Vladimir Putin' s gain.
             Already the largest country on the planet, Russia stands to gain
             access to shipping routes and energy reserves, and a strategic
             military advantage from the opening of the Arctic. Along the Russian
             coastline, which makes up more than half the Arctic total, winds
             and currents push old ice away from potential shipping lanes and
             prevent the build-up of thicker, multi-year ice that would leave
             other parts of the Arctic impassable for longer periods.
             """))

Here are the identified fallacies separated by ';':

guilt by association; hasty generalization
